# Task
Process PDF documents in the "content/" folder, performing page-level segmentation, text extraction with OCR and cleanup, table detection and extraction into a specified JSON format with summaries, and image detection and preprocessing with classification and metadata annotation. Combine the results into a structured output.

## Setup environment

### Subtask:
Install necessary libraries like PyMuPDF, PDFPlumber, pytesseract, pandas, and potentially Camelot or ChartOCR if needed for advanced table/chart processing.


**Reasoning**:
Install the required Python libraries for PDF processing, text extraction, and data manipulation using pip.



In [1]:
%pip install pymupdf pdfplumber pytesseract pandas camelot-py opencv-python scikit-image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 127.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 106.0 MB/s eta 0:00:00


## Load documents

### Subtask:
Iterate through PDF documents in the specified folder (`content/`).


**Reasoning**:
Import the os module, define the folder path, list files in the folder, and filter for PDF files.



In [2]:
import os

pdf_folder_path = "/content/"
os.makedirs(pdf_folder_path, exist_ok=True)

all_files = os.listdir(pdf_folder_path)
pdf_files = [os.path.join(pdf_folder_path, file) for file in all_files if file.endswith(".pdf")]

print(f"Found {len(pdf_files)} PDF files in {pdf_folder_path}")
print(pdf_files)

Found 6 PDF files in /content/
['/content/T-REC-D.1140-202008-I!!PDF-E.pdf', '/content/T-REC-E.812-202009-I!Amd1!PDF-E.pdf', '/content/T-REC-E.812-202005-I!!PDF-E.pdf', '/content/T-REC-D.609-202502-P!!PDF-E.pdf', '/content/T-REC-E.806-201906-I!!PDF-E.pdf', '/content/T-REC-E.811-201703-I!!PDF-E.pdf']


## Page-level segmentation

### Subtask:
For each document, split it into page objects using a library like PyMuPDF or PDFPlumber.


**Reasoning**:
Iterate through the list of pdf_files and open each document using fitz.open().



In [4]:
import fitz

document_pages = {}
for pdf_path in pdf_files:
    try:
        doc = fitz.open(pdf_path)
        document_pages[pdf_path] = [page for page in doc]
        print(f"Successfully processed {pdf_path} with {len(doc)} pages.")
        doc.close()
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")

print(f"\nProcessed documents: {list(document_pages.keys())}")

Successfully processed /content/T-REC-D.1140-202008-I!!PDF-E.pdf with 12 pages.
Successfully processed /content/T-REC-E.812-202009-I!Amd1!PDF-E.pdf with 38 pages.
Successfully processed /content/T-REC-E.812-202005-I!!PDF-E.pdf with 28 pages.
Successfully processed /content/T-REC-D.609-202502-P!!PDF-E.pdf with 9 pages.
Successfully processed /content/T-REC-E.806-201906-I!!PDF-E.pdf with 26 pages.
Successfully processed /content/T-REC-E.811-201703-I!!PDF-E.pdf with 36 pages.

Processed documents: ['/content/T-REC-D.1140-202008-I!!PDF-E.pdf', '/content/T-REC-E.812-202009-I!Amd1!PDF-E.pdf', '/content/T-REC-E.812-202005-I!!PDF-E.pdf', '/content/T-REC-D.609-202502-P!!PDF-E.pdf', '/content/T-REC-E.806-201906-I!!PDF-E.pdf', '/content/T-REC-E.811-201703-I!!PDF-E.pdf']


## Metadata extraction

### Subtask:
Extract page numbers and attempt to identify section headings based on heuristics (font size) or a Table of Contents.


**Reasoning**:
Iterate through each document and its pages, extract page numbers, and attempt to identify section headings based on font size heuristics.



In [5]:
page_info = {}

for doc_path, pages in document_pages.items():
    page_info[doc_path] = []
    for page_num, page in enumerate(pages):
        page_data = {"page_number": page_num + 1, "potential_headings": []}
        try:
            # Extract text blocks with font information
            text_blocks = page.get_text("dict")["blocks"]

            # Find the most common font size on the page as a baseline
            font_sizes = []
            for block in text_blocks:
                if block["type"] == 0: # text block
                    for line in block["lines"]:
                        for span in line["spans"]:
                            font_sizes.append(span["size"])

            if font_sizes:
                baseline_font_size = max(set(font_sizes), key=font_sizes.count)
            else:
                baseline_font_size = None

            # Identify potential headings based on font size heuristic
            if baseline_font_size is not None:
                for block in text_blocks:
                    if block["type"] == 0: # text block
                        for line in block["lines"]:
                            for span in line["spans"]:
                                # Heuristic: Text with significantly larger font size
                                if span["size"] > baseline_font_size * 1.2: # Adjust multiplier as needed
                                    page_data["potential_headings"].append(span["text"].strip())

        except Exception as e:
            print(f"Error processing page {page_num + 1} of {doc_path}: {e}")

        page_info[doc_path].append(page_data)

for doc_path, info in page_info.items():
    print(f"Processing results for {doc_path}:")
    for page_data in info:
        print(f"  Page {page_data['page_number']}:")
        if page_data["potential_headings"]:
            print("    Potential Headings:")
            for heading in page_data["potential_headings"]:
                print(f"      - {heading}")
        else:
            print("    No potential headings found.")


Error processing page 1 of /content/T-REC-D.1140-202008-I!!PDF-E.pdf: page is None
Error processing page 2 of /content/T-REC-D.1140-202008-I!!PDF-E.pdf: page is None
Error processing page 3 of /content/T-REC-D.1140-202008-I!!PDF-E.pdf: page is None
Error processing page 4 of /content/T-REC-D.1140-202008-I!!PDF-E.pdf: page is None
Error processing page 5 of /content/T-REC-D.1140-202008-I!!PDF-E.pdf: page is None
Error processing page 6 of /content/T-REC-D.1140-202008-I!!PDF-E.pdf: page is None
Error processing page 7 of /content/T-REC-D.1140-202008-I!!PDF-E.pdf: page is None
Error processing page 8 of /content/T-REC-D.1140-202008-I!!PDF-E.pdf: page is None
Error processing page 9 of /content/T-REC-D.1140-202008-I!!PDF-E.pdf: page is None
Error processing page 10 of /content/T-REC-D.1140-202008-I!!PDF-E.pdf: page is None
Error processing page 11 of /content/T-REC-D.1140-202008-I!!PDF-E.pdf: page is None
Error processing page 12 of /content/T-REC-D.1140-202008-I!!PDF-E.pdf: page is None
E

**Reasoning**:
The previous code failed because the page objects in `document_pages` became None after the document was closed in the previous step. Need to re-open the documents within the loop to access the page objects correctly and retry the heading extraction.



In [6]:
import fitz

page_info = {}

for doc_path in document_pages.keys():
    page_info[doc_path] = []
    try:
        doc = fitz.open(doc_path) # Re-open the document
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num) # Load the page
            page_data = {"page_number": page_num + 1, "potential_headings": []}
            try:
                # Extract text blocks with font information
                text_blocks = page.get_text("dict")["blocks"]

                # Find the most common font size on the page as a baseline
                font_sizes = []
                for block in text_blocks:
                    if block["type"] == 0: # text block
                        for line in block["lines"]:
                            for span in line["spans"]:
                                font_sizes.append(span["size"])

                if font_sizes:
                    baseline_font_size = max(set(font_sizes), key=font_sizes.count)
                else:
                    baseline_font_size = None

                # Identify potential headings based on font size heuristic
                if baseline_font_size is not None:
                    for block in text_blocks:
                        if block["type"] == 0: # text block
                            for line in block["lines"]:
                                for span in line["spans"]:
                                    # Heuristic: Text with significantly larger font size
                                    if span["size"] > baseline_font_size * 1.2: # Adjust multiplier as needed
                                        # Check if the text is at the beginning of a line/block
                                        # This is a simple check and might need refinement
                                        if span["origin"][0] < 50: # Assuming text starting near the left margin is a potential heading
                                            page_data["potential_headings"].append(span["text"].strip())

            except Exception as e:
                print(f"Error processing page {page_num + 1} of {doc_path}: {e}")

            page_info[doc_path].append(page_data)
        doc.close() # Close the document after processing
    except Exception as e:
        print(f"Error opening or processing {doc_path}: {e}")


for doc_path, info in page_info.items():
    print(f"Processing results for {doc_path}:")
    for page_data in info:
        print(f"  Page {page_data['page_number']}:")
        if page_data["potential_headings"]:
            print("    Potential Headings:")
            for heading in page_data["potential_headings"]:
                print(f"      - {heading}")
        else:
            print("    No potential headings found.")

Processing results for /content/T-REC-D.1140-202008-I!!PDF-E.pdf:
  Page 1:
    No potential headings found.
  Page 2:
    No potential headings found.
  Page 3:
    No potential headings found.
  Page 4:
    No potential headings found.
  Page 5:
    No potential headings found.
  Page 6:
    No potential headings found.
  Page 7:
    No potential headings found.
  Page 8:
    No potential headings found.
  Page 9:
    No potential headings found.
  Page 10:
    No potential headings found.
  Page 11:
    No potential headings found.
  Page 12:
    No potential headings found.
Processing results for /content/T-REC-E.812-202009-I!Amd1!PDF-E.pdf:
  Page 1:
    No potential headings found.
  Page 2:
    No potential headings found.
  Page 3:
    No potential headings found.
  Page 4:
    No potential headings found.
  Page 5:
    No potential headings found.
  Page 6:
    No potential headings found.
  Page 7:
    No potential headings found.
  Page 8:
    No potential headings found.
  

## Text extraction & cleanup

### Subtask:
Implement OCR for pages with non-selectable text using pytesseract. Clean up extracted text by normalizing whitespace, removing headers/footers (if detectable with heuristics), and handling hyphenation artifacts. Segment text into logical blocks (paragraphs, lists).


**Reasoning**:
Implement OCR for pages with non-selectable text, clean the extracted text by normalizing whitespace, handling hyphenation, and attempting to remove headers/footers, and segment the text into logical blocks.



In [7]:
import pytesseract
import numpy as np
from PIL import Image
import re
import io

# Function to clean text
def clean_text(text):
    # Normalize whitespace: replace multiple whitespace characters with a single space
    text = re.sub(r'\s+', ' ', text).strip()
    # Handle hyphenation artifacts: join hyphenated words split across lines
    text = re.sub(r'(\w+)-\s+(\w+)', r'\1\2', text)
    return text

# Simple heuristic to detect potential header/footer lines
def is_header_footer(line, page_height, top_margin=50, bottom_margin=50):
    # Check if the line is within the top or bottom margin
    # This is a very basic heuristic and might not be accurate for all documents
    y0, y1 = line['bbox'][1], line['bbox'][3]
    if y0 < top_margin or y1 > page_height - bottom_margin:
        # Further checks could be added, e.g., check for repeating text
        return True
    return False

for doc_path, pages in document_pages.items():
    page_info[doc_path] = [] # Re-initialize page_info for this document
    try:
        doc = fitz.open(doc_path)
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            page_data = {"page_number": page_num + 1, "text_blocks": [], "potential_headings": []} # Add text_blocks field

            text = page.get_text()

            # Heuristic for non-selectable text: empty text or very short text
            if not text or len(text.strip()) < 50: # Adjust threshold as needed
                print(f"Performing OCR on page {page_num + 1} of {doc_path}")
                # Convert page to image for OCR
                pix = page.get_pixmap()
                img = Image.open(io.BytesIO(pix.tobytes("png")))
                text = pytesseract.image_to_string(img)

            cleaned_text = clean_text(text)

            # Attempt to remove headers/footers based on simple heuristic
            # This is done after cleaning but before segmentation
            page_height = page.rect.height
            cleaned_lines = cleaned_text.split('\n')
            filtered_lines = []
            for line_num, line_text in enumerate(cleaned_lines):
                # This heuristic is difficult to apply reliably without detailed layout info after OCR
                # For now, we'll skip the header/footer removal based on position for OCR text
                # and rely on pattern matching or content-based filtering if needed later.
                filtered_lines.append(line_text)

            # Re-join lines after potential filtering for segmentation
            text_for_segmentation = '\n'.join(filtered_lines)

            # Segment text into logical blocks (paragraphs)
            # Split by multiple newline characters
            text_blocks = [block.strip() for block in re.split(r'\n\s*\n', text_for_segmentation) if block.strip()]

            page_data["text_blocks"] = text_blocks

            # Re-extract potential headings using the previous logic (optional, depends on desired output structure)
            # Keeping the previous heading extraction logic separate for now
            try:
                text_blocks_with_info = page.get_text("dict")["blocks"]
                font_sizes = []
                for block in text_blocks_with_info:
                    if block["type"] == 0: # text block
                        for line in block["lines"]:
                            for span in line["spans"]:
                                font_sizes.append(span["size"])

                if font_sizes:
                    baseline_font_size = max(set(font_sizes), key=font_sizes.count)
                else:
                    baseline_font_size = None

                if baseline_font_size is not None:
                    for block in text_blocks_with_info:
                        if block["type"] == 0: # text block
                            for line in block["lines"]:
                                for span in line["spans"]:
                                    if span["size"] > baseline_font_size * 1.2:
                                        if span["origin"][0] < 50:
                                            page_data["potential_headings"].append(span["text"].strip())
            except Exception as e:
                 print(f"Error re-extracting headings for page {page_num + 1} of {doc_path}: {e}")


            page_info[doc_path].append(page_data)

        doc.close()
    except Exception as e:
        print(f"Error processing {doc_path}: {e}")

# Print sample output
for doc_path, info in page_info.items():
    print(f"Processed text blocks for {doc_path}:")
    for page_data in info[:2]: # Print info for first 2 pages as sample
        print(f"  Page {page_data['page_number']}:")
        for i, block in enumerate(page_data["text_blocks"][:3]): # Print first 3 text blocks
            print(f"    Block {i+1}: {block[:200]}...") # Print first 200 chars
        if page_data["potential_headings"]:
            print("    Potential Headings:")
            for heading in page_data["potential_headings"]:
                print(f"      - {heading}")


Performing OCR on page 6 of /content/T-REC-D.1140-202008-I!!PDF-E.pdf
Performing OCR on page 37 of /content/T-REC-E.812-202009-I!Amd1!PDF-E.pdf
Performing OCR on page 27 of /content/T-REC-E.812-202005-I!!PDF-E.pdf
Performing OCR on page 6 of /content/T-REC-E.806-201906-I!!PDF-E.pdf
Performing OCR on page 25 of /content/T-REC-E.806-201906-I!!PDF-E.pdf
Performing OCR on page 35 of /content/T-REC-E.811-201703-I!!PDF-E.pdf
Processed text blocks for /content/T-REC-D.1140-202008-I!!PDF-E.pdf:
  Page 1:
    Block 1: I n t e r n a t i o n a l T e l e c o m m u n i c a t i o n U n i o n ITU-T D.1140/X.1261 TELECOMMUNICATION STANDARDIZATION SECTOR OF ITU (08/2020) SERIES D: TARIFF AND ACCOUNTING PRINCIPLES AND INTER...
  Page 2:
    Block 1: ITU-T D-SERIES RECOMMENDATIONS TARIFF AND ACCOUNTING PRINCIPLES AND INTERNATIONAL TELECOMMUNICATION/ICT ECONOMIC AND POLICY ISSUES TERMS AND DEFINITIONS D.0 GENERAL TARIFF PRINCIPLES Private leased te...
Processed text blocks for /content/T-REC-E.812-202009-

## Table detection & extraction

### Subtask:
Detect tables using PDFPlumber or Camelot. Extract table data into DataFrames. Normalize cell contents. Serialize each table into the specified JSON format. Generate a natural-language summary for each table (this might require further NLP capabilities or template-based generation).


**Reasoning**:
Iterate through each page, detect tables using pdfplumber, extract data into DataFrames, normalize cell content, convert to JSON-compatible format, and store in `page_info` along with a basic summary.



In [8]:
import pdfplumber
import pandas as pd
import json

# Function to normalize cell content
def normalize_cell_content(cell_text):
    if cell_text is None:
        return ""
    # Remove leading/trailing whitespace
    cell_text = cell_text.strip()
    # Replace multiple spaces with a single space
    cell_text = re.sub(r'\s+', ' ', cell_text)
    return cell_text

# Function to generate a basic table summary
def generate_table_summary(df, table_index):
    if df.empty:
        return f"Table {table_index + 1}: Empty table."
    num_rows, num_cols = df.shape
    summary = f"Table {table_index + 1}: A table with {num_rows} rows and {num_cols} columns."
    if num_cols > 0:
        summary += f" Columns include: {', '.join(df.columns[:5])}" # List up to 5 column names
        if num_cols > 5:
            summary += "..."
    return summary

for doc_path, pages_data in list(page_info.items()): # Iterate through a copy of the keys
    try:
        with pdfplumber.open(doc_path) as pdf:
            for page_num, page_data in enumerate(pages_data):
                page = pdf.pages[page_num]
                detected_tables = page.extract_tables()

                page_data['tables'] = [] # Add a new key for tables

                for table_index, table_data in enumerate(detected_tables):
                    if table_data:
                        try:
                            # Convert to DataFrame
                            df = pd.DataFrame(table_data[1:], columns=table_data[0]) # Assuming first row is header

                            # Normalize cell content
                            df = df.applymap(normalize_cell_content)

                            # Convert DataFrame to list of dictionaries (JSON format)
                            table_json_data = df.to_dict('records')

                            # Generate summary
                            table_summary = generate_table_summary(df, table_index)

                            # Store in page_data
                            page_data['tables'].append({
                                'data': table_json_data,
                                'summary': table_summary
                            })
                        except Exception as e:
                            print(f"Error processing table {table_index + 1} on page {page_num + 1} of {doc_path}: {e}")

    except Exception as e:
        print(f"Error opening or processing {doc_path} with pdfplumber: {e}")

# Print sample output for tables
for doc_path, info in page_info.items():
    print(f"\nProcessed tables for {doc_path}:")
    for page_data in info:
        if page_data.get('tables'):
            print(f"  Page {page_data['page_number']}:")
            for table_info in page_data['tables']:
                print(f"    Table Summary: {table_info['summary']}")
                # Print first few rows of table data as sample
                print(f"    Sample Data: {json.dumps(table_info['data'][:2], indent=2)}...") # Print first 2 rows



/tmp/ipython-input-8-884452526.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(normalize_cell_content)
/tmp/ipython-input-8-884452526.py:46: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  table_json_data = df.to_dict('records')
/tmp/ipython-input-8-884452526.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(normalize_cell_content)
/tmp/ipython-input-8-884452526.py:46: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  table_json_data = df.to_dict('records')


Error processing table 1 on page 2 of /content/T-REC-D.1140-202008-I!!PDF-E.pdf: sequence item 1: expected str instance, NoneType found


/tmp/ipython-input-8-884452526.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(normalize_cell_content)
/tmp/ipython-input-8-884452526.py:46: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  table_json_data = df.to_dict('records')


Error processing table 1 on page 11 of /content/T-REC-D.1140-202008-I!!PDF-E.pdf: sequence item 1: expected str instance, NoneType found


/tmp/ipython-input-8-884452526.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(normalize_cell_content)
/tmp/ipython-input-8-884452526.py:46: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  table_json_data = df.to_dict('records')
/tmp/ipython-input-8-884452526.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(normalize_cell_content)
/tmp/ipython-input-8-884452526.py:46: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  table_json_data = df.to_dict('records')


...

/tmp/ipython-input-8-884452526.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(normalize_cell_content)


Error processing table 1 on page 22 of /content/T-REC-E.811-201703-I!!PDF-E.pdf: sequence item 3: expected str instance, NoneType found


/tmp/ipython-input-8-884452526.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(normalize_cell_content)
/tmp/ipython-input-8-884452526.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(normalize_cell_content)
/tmp/ipython-input-8-884452526.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(normalize_cell_content)
/tmp/ipython-input-8-884452526.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(normalize_cell_content)


Error processing table 3 on page 32 of /content/T-REC-E.811-201703-I!!PDF-E.pdf: sequence item 3: expected str instance, NoneType found

Processed tables for /content/T-REC-D.1140-202008-I!!PDF-E.pdf:
  Page 3:
    Table Summary: Table 1: Empty table.
    Sample Data: []...

Processed tables for /content/T-REC-E.812-202009-I!Amd1!PDF-E.pdf:
  Page 3:
    Table Summary: Table 1: A table with 1 rows and 15 columns. Columns include: , 1.0, , , ITU-T E.812...
    Sample Data: [
  {
    "": "",
    "1.0": "1.1",
    "ITU-T E.812": "ITU-T E.812 (2020) Amd. 1",
    "2020-05-29": "2020-09-11",
    "12": "12",
    "11.1002/1000/14272": "11.1002/1000/14489"
  }
]...
  Page 20:
    Table Summary: Table 1: A table with 3 rows and 3 columns. Columns include: Type of server
hosting, Characteristics, Typical services
    Sample Data: [
  {
    "Type of server\nhosting": "Network point-of- presence (PoP)",
    "Characteristics": "Servers are located in the network infrastructure of service providers o

/tmp/ipython-input-8-884452526.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(normalize_cell_content)
/tmp/ipython-input-8-884452526.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(normalize_cell_content)
/tmp/ipython-input-8-884452526.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(normalize_cell_content)
/tmp/ipython-input-8-884452526.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(normalize_cell_content)


**Reasoning**:
The previous attempt encountered errors when processing tables, specifically "sequence item 1: expected str instance, NoneType found". This indicates that some cells in the extracted tables might contain `None` instead of strings, causing issues when creating the DataFrame or normalizing cell content. Additionally, there are `FutureWarning` and `UserWarning` about `applymap` deprecation and non-unique columns when converting to dictionary. The code needs to handle potential `None` values gracefully during DataFrame creation and normalization, use the recommended `map` instead of `applymap`, and address the non-unique columns issue if necessary (although `to_dict('records')` usually handles this by keeping the last column with a duplicate name). Let's focus on handling the `None` values during DataFrame creation and cell normalization, and use `map` for normalization.



In [9]:
import pdfplumber
import pandas as pd
import json
import re

# Function to normalize cell content
def normalize_cell_content(cell_text):
    if cell_text is None:
        return ""
    # Ensure the input is a string before applying string methods
    cell_text = str(cell_text)
    # Remove leading/trailing whitespace
    cell_text = cell_text.strip()
    # Replace multiple spaces with a single space
    cell_text = re.sub(r'\s+', ' ', cell_text)
    return cell_text

# Function to generate a basic table summary
def generate_table_summary(df, table_index):
    if df.empty:
        return f"Table {table_index + 1}: Empty table."
    num_rows, num_cols = df.shape
    summary = f"Table {table_index + 1}: A table with {num_rows} rows and {num_cols} columns."
    if num_cols > 0:
        # Get unique column names for the summary
        unique_cols = []
        seen = set()
        for col in df.columns:
            if col not in seen:
                unique_cols.append(col)
                seen.add(col)
        summary += f" Columns include: {', '.join(unique_cols[:5])}" # List up to 5 unique column names
        if num_cols > 5:
            summary += "..."
    return summary

for doc_path, pages_data in list(page_info.items()): # Iterate through a copy of the keys
    try:
        with pdfplumber.open(doc_path) as pdf:
            # Ensure page_info has an entry for this document if it was somehow missed
            if doc_path not in page_info:
                 page_info[doc_path] = []
                 # Populate with basic page data if missing (unlikely given previous steps)
                 for page_num in range(len(pdf.pages)):
                      page_info[doc_path].append({"page_number": page_num + 1, "text_blocks": [], "potential_headings": [], "tables": []})
            else:
                # Ensure each page entry has a 'tables' key
                for page_data in pages_data:
                    if 'tables' not in page_data:
                        page_data['tables'] = []


            for page_num, page_data in enumerate(page_info[doc_path]): # Iterate using page_info index
                if page_num < len(pdf.pages): # Ensure page exists in pdfplumber object
                    page = pdf.pages[page_num]
                    detected_tables = page.extract_tables()

                    for table_index, table_data in enumerate(detected_tables):
                        if table_data and len(table_data) > 1: # Ensure table is not empty and has at least a header and one row
                            try:
                                # Handle potential None values in header row
                                header = [normalize_cell_content(h) for h in table_data[0]]

                                # Handle potential None values in data rows
                                processed_data = []
                                for row in table_data[1:]:
                                    processed_row = [normalize_cell_content(cell) for cell in row]
                                    # Ensure row has the same number of columns as the header
                                    while len(processed_row) < len(header):
                                        processed_row.append("") # Pad with empty strings if necessary
                                    processed_data.append(processed_row[:len(header)]) # Truncate if too many columns

                                if not processed_data: # Handle case where data rows are empty after processing
                                     df = pd.DataFrame(columns=header)
                                else:
                                     df = pd.DataFrame(processed_data, columns=header)


                                # Normalize cell content using map (recommended)
                                # df = df.map(normalize_cell_content) # Apply to all cells - this can be slow and might not be needed after row processing


                                # Convert DataFrame to list of dictionaries (JSON format)
                                # Use to_dict('records'). Address potential duplicate columns in summary generation.
                                table_json_data = df.to_dict('records')

                                # Generate summary
                                table_summary = generate_table_summary(df, table_index)

                                # Store in page_data
                                page_data['tables'].append({
                                    'data': table_json_data,
                                    'summary': table_summary
                                })
                            except Exception as e:
                                print(f"Error processing table {table_index + 1} on page {page_num + 1} of {doc_path}: {e}")
                        elif table_data and len(table_data) <= 1:
                             print(f"Skipping potential empty table {table_index + 1} on page {page_num + 1} of {doc_path}")


    except Exception as e:
        print(f"Error opening or processing {doc_path} with pdfplumber: {e}")

# Print sample output for tables
for doc_path, info in page_info.items():
    print(f"\nProcessed tables for {doc_path}:")
    for page_data in info:
        if page_data.get('tables'):
            print(f"  Page {page_data['page_number']}:")
            for table_info in page_data['tables']:
                print(f"    Table Summary: {table_info['summary']}")
                # Print first few rows of table data as sample
                print(f"    Sample Data: {json.dumps(table_info['data'][:2], indent=2)}...") # Print first 2 rows


/tmp/ipython-input-9-2325800251.py:85: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  table_json_data = df.to_dict('records')


Skipping potential empty table 1 on page 3 of /content/T-REC-D.1140-202008-I!!PDF-E.pdf


/tmp/ipython-input-9-2325800251.py:85: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  table_json_data = df.to_dict('records')
/tmp/ipython-input-9-2325800251.py:85: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  table_json_data = df.to_dict('records')
/tmp/ipython-input-9-2325800251.py:85: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  table_json_data = df.to_dict('records')
/tmp/ipython-input-9-2325800251.py:85: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  table_json_data = df.to_dict('records')
/tmp/ipython-input-9-2325800251.py:85: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  table_json_data = df.to_dict('records')
/tmp/ipython-input-9-2325800251.py:85: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  table_json_data = df.to_dict('records')


Skipping potential empty table 1 on page 3 of /content/T-REC-E.812-202005-I!!PDF-E.pdf


/tmp/ipython-input-9-2325800251.py:85: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  table_json_data = df.to_dict('records')
/tmp/ipython-input-9-2325800251.py:85: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  table_json_data = df.to_dict('records')
/tmp/ipython-input-9-2325800251.py:85: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  table_json_data = df.to_dict('records')


...

Skipping potential empty table 1 on page 3 of /content/T-REC-E.806-201906-I!!PDF-E.pdf


/tmp/ipython-input-9-2325800251.py:85: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  table_json_data = df.to_dict('records')


Skipping potential empty table 1 on page 3 of /content/T-REC-E.811-201703-I!!PDF-E.pdf


/tmp/ipython-input-9-2325800251.py:85: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  table_json_data = df.to_dict('records')
/tmp/ipython-input-9-2325800251.py:85: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  table_json_data = df.to_dict('records')
/tmp/ipython-input-9-2325800251.py:85: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  table_json_data = df.to_dict('records')
/tmp/ipython-input-9-2325800251.py:85: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  table_json_data = df.to_dict('records')



Processed tables for /content/T-REC-D.1140-202008-I!!PDF-E.pdf:
  Page 2:
    Table Summary: Table 1: A table with 3 rows and 4 columns. Columns include: TERMS AND DEFINITIONS D.0 GENERAL TARIFF PRINCIPLES Private leased telecommunication facilities D.1–D.9 Tariff principles applying to data communication services over dedicated public data networks D.10–D.39 Charging and accounting in the international public telegram service D.40–D.44 Charging and accounting in the international telemessage service D.45–D.49 Principles applicable to GII-Internet D.50–D.59 Charging and accounting in the international telex service D.60–D.69 Charging and accounting in the international facsimile service D.70–D.75 Charging and accounting in the international videotex service D.76–D.79 Charging and accounting in the international phototelegraph service D.80–D.89 Charging and accounting in the mobile services D.90–D.99 Charging and accounting in the international telephone service D.100–D.159 Drawing up 

**Reasoning**:
The previous code successfully extracted tables and handled most errors, but the `UserWarning` about non-unique columns when converting to a dictionary persists. While `to_dict('records')` handles this by keeping the last occurring column with a duplicate name, it's better practice to ensure unique column names before conversion. A simple approach is to append a suffix to duplicate column names. Also, ensure that the table data is correctly appended to the existing `page_info` list for each page, not overwriting it. The current code re-initializes `page_info` for each document, which is incorrect. Let's fix this and handle duplicate column names.



In [10]:
import pdfplumber
import pandas as pd
import json
import re
from collections import defaultdict

# Function to normalize cell content
def normalize_cell_content(cell_text):
    if cell_text is None:
        return ""
    # Ensure the input is a string before applying string methods
    cell_text = str(cell_text)
    # Remove leading/trailing whitespace
    cell_text = cell_text.strip()
    # Replace multiple spaces with a single space
    cell_text = re.sub(r'\s+', ' ', cell_text)
    return cell_text

# Function to generate a basic table summary
def generate_table_summary(df, table_index):
    if df.empty:
        return f"Table {table_index + 1}: Empty table."
    num_rows, num_cols = df.shape
    summary = f"Table {table_index + 1}: A table with {num_rows} rows and {num_cols} columns."
    if num_cols > 0:
        # Get unique column names for the summary
        unique_cols = []
        seen = set()
        for col in df.columns:
            if col not in seen:
                unique_cols.append(col)
                seen.add(col)
        summary += f" Columns include: {', '.join(unique_cols[:5])}" # List up to 5 unique column names
        if num_cols > 5:
            summary += "..."
    return summary

# Function to make column names unique
def make_columns_unique(columns):
    seen = defaultdict(int)
    new_columns = []
    for col in columns:
        if seen[col] > 0:
            new_columns.append(f"{col}_{seen[col]}")
        else:
            new_columns.append(col)
        seen[col] += 1
    return new_columns


# Iterate through a copy of the keys to avoid issues if the dict is modified during iteration
for doc_path in list(page_info.keys()):
    try:
        with pdfplumber.open(doc_path) as pdf:
            # Ensure page_info has an entry for this document and that each page has a 'tables' key
            if doc_path not in page_info:
                 # This case should ideally not happen given previous steps, but as a safeguard
                 page_info[doc_path] = []
                 for page_num in range(len(pdf.pages)):
                      page_info[doc_path].append({"page_number": page_num + 1, "text_blocks": [], "potential_headings": [], "tables": []})
            else:
                # Ensure each page entry has a 'tables' key and is a list
                for page_data in page_info[doc_path]:
                    if 'tables' not in page_data or not isinstance(page_data['tables'], list):
                        page_data['tables'] = []


            # Iterate through page_info which already contains page numbers and other data
            for page_num, page_data in enumerate(page_info[doc_path]):
                # Ensure we don't go out of bounds for the pdfplumber pages
                if page_num < len(pdf.pages):
                    page = pdf.pages[page_num]
                    detected_tables = page.extract_tables()

                    for table_index, table_data in enumerate(detected_tables):
                        # Ensure table is not empty and has at least a header row
                        if table_data and len(table_data) >= 1:
                            try:
                                # Handle potential None values in header row and normalize
                                header = [normalize_cell_content(h) for h in table_data[0]]

                                # Make header columns unique
                                unique_header = make_columns_unique(header)

                                # Handle potential None values in data rows and normalize
                                processed_data = []
                                for row in table_data[1:]:
                                    processed_row = [normalize_cell_content(cell) for cell in row]
                                    # Ensure row has the same number of columns as the header, padding or truncating
                                    while len(processed_row) < len(unique_header):
                                        processed_row.append("") # Pad with empty strings if necessary
                                    processed_data.append(processed_row[:len(unique_header)]) # Truncate if too many columns

                                # Create DataFrame with unique columns
                                if not processed_data and not unique_header: # Handle completely empty table data
                                     df = pd.DataFrame()
                                elif not processed_data: # Handle table with header but no data rows
                                     df = pd.DataFrame(columns=unique_header)
                                else:
                                     df = pd.DataFrame(processed_data, columns=unique_header)


                                # Convert DataFrame to list of dictionaries (JSON format)
                                table_json_data = df.to_dict('records')

                                # Generate summary
                                table_summary = generate_table_summary(df, table_index)

                                # Store in page_data's 'tables' list
                                page_data['tables'].append({
                                    'data': table_json_data,
                                    'summary': table_summary
                                })
                            except Exception as e:
                                print(f"Error processing table {table_index + 1} on page {page_num + 1} of {doc_path}: {e}")
                        elif table_data and len(table_data) < 1: # This case should not be reachable with len(table_data) >= 1 check, but as a safeguard
                             print(f"Skipping potential empty table {table_index + 1} on page {page_num + 1} of {doc_path} (no header)")


    except Exception as e:
        print(f"Error opening or processing {doc_path} with pdfplumber: {e}")

# Print sample output for tables
for doc_path, info in page_info.items():
    print(f"\nProcessed tables for {doc_path}:")
    for page_data in info:
        if page_data.get('tables'):
            print(f"  Page {page_data['page_number']}:")
            for table_info in page_data['tables']:
                print(f"    Table Summary: {table_info['summary']}")
                # Print first few rows of table data as sample
                print(f"    Sample Data: {json.dumps(table_info['data'][:2], indent=2)}...") # Print first 2 rows



Processed tables for /content/T-REC-D.1140-202008-I!!PDF-E.pdf:
  Page 2:
    Table Summary: Table 1: A table with 3 rows and 4 columns. Columns include: TERMS AND DEFINITIONS D.0 GENERAL TARIFF PRINCIPLES Private leased telecommunication facilities D.1–D.9 Tariff principles applying to data communication services over dedicated public data networks D.10–D.39 Charging and accounting in the international public telegram service D.40–D.44 Charging and accounting in the international telemessage service D.45–D.49 Principles applicable to GII-Internet D.50–D.59 Charging and accounting in the international telex service D.60–D.69 Charging and accounting in the international facsimile service D.70–D.75 Charging and accounting in the international videotex service D.76–D.79 Charging and accounting in the international phototelegraph service D.80–D.89 Charging and accounting in the mobile services D.90–D.99 Charging and accounting in the international telephone service D.100–D.159 Drawing up 

## Image detection & preprocessing

### Subtask:
Identify images using PyMuPDF. Extract images to files. Implement a simple image classification (e.g., using a pre-trained lightweight CNN or rule-based approach). (Optional) Integrate ChartOCR if needed. Annotate images with metadata.


**Reasoning**:
Iterate through each page of each document, extract images using PyMuPDF, save them to temporary files, perform a simple rule-based classification, and annotate the images with metadata in `page_info`.



In [12]:
import fitz
import io
from PIL import Image
import os

# Directory to save temporary images
temp_image_dir = "temp_images"
os.makedirs(temp_image_dir, exist_ok=True)

for doc_path in list(document_pages.keys()):
    try:
        doc = fitz.open(doc_path)
        # Ensure page_info has an entry for this document
        if doc_path not in page_info:
             page_info[doc_path] = []
             for page_num in range(doc.page_count):
                  page_info[doc_path].append({"page_number": page_num + 1, "text_blocks": [], "potential_headings": [], "tables": [], "images": []})
        else:
            # Ensure each page entry has an 'images' key and is a list
            for page_data in page_info[doc_path]:
                if 'images' not in page_data or not isinstance(page_data['images'], list):
                    page_data['images'] = []


        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            images = page.get_images(full=True) # Extract images with full data

            # Find the corresponding page_data in page_info
            current_page_data = None
            for pd in page_info[doc_path]:
                if pd["page_number"] == page_num + 1:
                    current_page_data = pd
                    break

            if current_page_data is None:
                # If page_data not found (shouldn't happen if page_info is populated correctly), add it
                current_page_data = {"page_number": page_num + 1, "text_blocks": [], "potential_headings": [], "tables": [], "images": []}
                page_info[doc_path].append(current_page_data)


            for img_index, img_info in enumerate(images):
                xref = img_info[0]
                base_image = doc.extract_image(xref)
                img_bytes = base_image["image"]
                img_ext = base_image["ext"]

                # Save image to a temporary file
                temp_img_path = os.path.join(temp_image_dir, f"{os.path.basename(doc_path)}_page{page_num+1}_img{img_index}.{img_ext}")
                try:
                    with open(temp_img_path, "wb") as f:
                        f.write(img_bytes)

                    # Load image to get dimensions for classification
                    img = Image.open(temp_img_path)
                    width, height = img.size

                    # Simple rule-based classification: based on aspect ratio and size
                    # This is a basic heuristic and may not be accurate
                    aspect_ratio = width / height if height > 0 else 0
                    image_type = 'other'
                    if width > 100 and height > 100: # Basic size check
                        if 0.8 < aspect_ratio < 1.2:
                            image_type = 'square_image'
                        elif aspect_ratio > 1.5 or aspect_ratio < 0.5:
                            image_type = 'wide_or_tall_image'
                        # More sophisticated rules or integration with ChartOCR would be needed for chart detection

                    # Get bounding box (PyMuPDF's get_images doesn't directly provide bbox, need to search)
                    # Searching for the image's position on the page is complex and not directly available
                    # from get_images(). For this task, we'll skip extracting the exact bbox from the page
                    # and only record that an image was found on the page. If bbox is crucial,
                    # a more advanced layout analysis would be required.
                    # Placeholder for bbox: Assuming the entire page for simplicity or skipping
                    # Let's try to find the image rect if possible, though it's not guaranteed to match
                    # images extracted by get_images(). This is a simplified approach.
                    image_bbox = None
                    try:
                        # This attempts to find image blocks on the page, which might correspond
                        # to the extracted images. Not a perfect match to get_images().
                        for block in page.get_text("dict")["blocks"]:
                             if block["type"] == 1: # Image block
                                  # Check if the image in this block corresponds to the extracted image (heuristic based on size)
                                  # This is a weak check and might be inaccurate
                                  if abs(block['bbox'][2] - block['bbox'][0]) > 10 and abs(block['bbox'][3] - block['bbox'][1]) > 10: # Basic size check for block
                                      image_bbox = block['bbox']
                                      break # Assuming the first found image block corresponds

                    except Exception as bbox_e:
                        print(f"Could not find image bbox on page {page_num + 1} of {doc_path}: {bbox_e}")
                        image_bbox = None # Explicitly set to None if search fails


                    # Annotate image with metadata
                    image_metadata = {
                        'image_index': img_index,
                        'original_xref': xref,
                        'file_path': temp_img_path,
                        'width': width,
                        'height': height,
                        'classification': image_type,
                        'bbox_on_page': image_bbox, # Use the found bbox or None
                        'page_number': page_num + 1
                    }
                    current_page_data['images'].append(image_metadata)

                except Exception as save_e:
                    print(f"Error saving or processing image {img_index} on page {page_num + 1} of {doc_path}: {save_e}")


        doc.close()
    except Exception as e:
        print(f"Error processing {doc_path} for images: {e}")

# Optional: Clean up temporary files after all processing (can be done at the end of the main task)
# import shutil
# shutil.rmtree(temp_image_dir)


# Print sample output for images
for doc_path, info in page_info.items():
    print(f"\nProcessed images for {doc_path}:")
    for page_data in info:
        if page_data.get('images'):
            print(f"  Page {page_data['page_number']}:")
            for img_info in page_data['images'][:3]: # Print info for first 3 images per page
                 print(f"    Image {img_info['image_index']}: Classification='{img_info['classification']}', Size={img_info['width']}x{img_info['height']}, BBox={img_info['bbox_on_page']}")



Processed images for /content/T-REC-D.1140-202008-I!!PDF-E.pdf:
  Page 1:
    Image 0: Classification='other', Size=161x67, BBox=(419.3999938964844, 748.5960083007812, 540.1500244140625, 798.8460083007812)
    Image 1: Classification='wide_or_tall_image', Size=122x843, BBox=(419.3999938964844, 748.5960083007812, 540.1500244140625, 798.8460083007812)
    Image 0: Classification='other', Size=161x67, BBox=(419.3999938964844, 748.5960083007812, 540.1500244140625, 798.8460083007812)

Processed images for /content/T-REC-E.812-202009-I!Amd1!PDF-E.pdf:
  Page 1:
    Image 0: Classification='other', Size=161x67, BBox=(419.3999938964844, 750.8460083007812, 540.1500244140625, 801.0960083007812)
    Image 1: Classification='wide_or_tall_image', Size=122x843, BBox=(419.3999938964844, 750.8460083007812, 540.1500244140625, 801.0960083007812)
    Image 0: Classification='other', Size=161x67, BBox=(419.3999938964844, 750.8460083007812, 540.1500244140625, 801.0960083007812)
  Page 10:
    Image 0: Cla

## Combine and structure output

### Subtask:
Combine the extracted text, tables, and image metadata for each page or document into a structured format (e.g., JSON or a custom object).


**Reasoning**:
Combine the extracted text blocks, potential headings, tables, and image metadata for each page and document into a structured dictionary format, as required by the subtask.



In [13]:
structured_output = {}

for doc_path, pages_data in page_info.items():
    structured_output[doc_path] = {
        "document_path": doc_path,
        "pages": []
    }
    for page_data in pages_data:
        structured_page_data = {
            "page_number": page_data.get("page_number"),
            "text_blocks": page_data.get("text_blocks", []),
            "potential_headings": page_data.get("potential_headings", []),
            "tables": page_data.get("tables", []),
            "images": page_data.get("images", [])
        }
        structured_output[doc_path]["pages"].append(structured_page_data)

# Print a sample of the structured output
import json

# Select a document to display (e.g., the first one)
if structured_output:
    first_doc_path = list(structured_output.keys())[0]
    print(f"Sample Structured Output for {first_doc_path}:")
    # Print a sample of the first few pages
    sample_output = structured_output[first_doc_path]
    sample_output["pages"] = sample_output["pages"][:2] # Limit to first 2 pages for sample
    print(json.dumps(sample_output, indent=2))
else:
    print("No structured output generated.")

Sample Structured Output for /content/T-REC-D.1140-202008-I!!PDF-E.pdf:
{
  "document_path": "/content/T-REC-D.1140-202008-I!!PDF-E.pdf",
  "pages": [
    {
      "page_number": 1,
      "text_blocks": [
        "I n t e r n a t i o n a l T e l e c o m m u n i c a t i o n U n i o n ITU-T D.1140/X.1261 TELECOMMUNICATION STANDARDIZATION SECTOR OF ITU (08/2020) SERIES D: TARIFF AND ACCOUNTING PRINCIPLES AND INTERNATIONAL TELECOMMUNICATION/ICT ECONOMIC AND POLICY ISSUES Recommendations for international telecommunication/ICT economic and policy issues \u2013 Economic and policy aspects of big data and digital identity in international telecommunications services and networks SERIES X: DATA NETWORKS, OPEN SYSTEM COMMUNICATIONS AND SECURITY Cyberspace security \u2013 Identity management Policy framework including principles for digital identity infrastructure Recommendation ITU-T D.1140/X.1261"
      ],
      "potential_headings": [],
      "tables": [],
      "images": [
        {
         

## Summary:

### Data Analysis Key Findings

*   The initial attempt to iterate through PDF documents failed because the specified directory did not exist. Creating the directory resolved this.
*   The first attempt at page-level processing using previously opened document objects failed because the objects were closed. Re-opening documents and loading pages individually resolved this issue.
*   The heuristic for identifying potential headings based on font size and position yielded limited and not always accurate results for the sample documents.
*   OCR was successfully implemented for pages with non-selectable text based on a heuristic of empty or short extracted text.
*   Text cleaning functions were implemented to normalize whitespace and handle hyphenation artifacts.
*   Table detection and extraction using `pdfplumber` were implemented, including normalization of cell content and handling of potential `None` values.
*   A function to make table column headers unique was successfully implemented to avoid issues during DataFrame conversion to dictionary format.
*   A basic natural-language summary was generated for each extracted table, including dimensions and sample column names.
*   Image detection and extraction using PyMuPDF were successful, with images saved to temporary files.
*   A simple rule-based image classification based on dimensions and aspect ratio was implemented.
*   Image metadata, including size, classification, and page number, was successfully annotated. An attempt to find the image bounding box on the page was made but noted as a potentially inaccurate simplified approach.
*   The final structured output successfully combined the extracted text blocks, potential headings, tables, and image metadata for each document and page into a nested dictionary format.

### Insights or Next Steps

*   Improve the heading detection logic by exploring more robust methods beyond simple font size heuristics, potentially involving analysis of the document's structure or a Table of Contents if available.
*   Enhance the image classification by integrating a more sophisticated model or library capable of identifying specific types of images, such as charts, figures, or photographs, potentially using ChartOCR for chart-specific analysis.


# Task
Process PDF documents in the "content/" folder to extract and normalize text, tables, images, and embedded audio, then chunk the content by modality, enrich with metadata, and generate multimodal embeddings.

## Embedded audio extraction & transcription

### Subtask:
Extract embedded audio streams from PDF (if supported) or from attachments. Convert audio to a uniform format (e.g., WAV, 16 kHz mono). Transcribe with a speech-to-text engine (e.g., Whisper). Segment transcription by speaker or time-chunk (e.g., every 15 sec). Store transcription alongside time-offset metadata.


**Reasoning**:
Import necessary libraries for audio processing and attachment handling, and iterate through the structured output to attempt audio extraction, conversion, transcription, and metadata storage. Given that direct embedded audio extraction from PDFs is complex and not universally supported by common Python libraries, this code will focus on checking for attachments and assuming they might be audio files, then proceed with transcription if found. It will also include placeholder comments for the audio extraction part due to the complexity and lack of standard library support.



**Reasoning**:
The previous command failed because the `whisper` library installed via `pip install whisper` is not the OpenAI Whisper model. The OpenAI Whisper model is available through the `openai-whisper` package. Additionally, the PyPDF2 embedded file access is still incorrect; for older versions it was `reader.attachments`, for newer versions `reader.embedded_files`. The traceback suggests an older version might be in use or there's another issue with accessing embedded files. Given the inconsistencies and potential version issues with PyPDF2 and embedded files, and the incorrect Whisper package, the best approach is to install the correct OpenAI Whisper package (`openai-whisper`) and acknowledge that embedded audio extraction from PDFs without attachments is complex and not fully supported by standard libraries. The code will focus on the transcription part using the correct Whisper package, but the audio extraction from PDFs will remain limited to checking for attachments using a more robust method if available or noting the difficulty. Let's try to install `openai-whisper` and update the PyPDF2 part based on typical usage.



In [15]:
%pip install openai-whisper PyPDF2 pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 19.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.0 M

## Embedded audio extraction & transcription

### Subtask:
Extract embedded audio streams from PDF (if supported) or from attachments. Convert audio to a uniform format (e.g., WAV, 16 kHz mono). Transcribe with a speech-to-text engine (e.g., Whisper). Segment transcription by speaker or time-chunk (e.g., every 15 sec). Store transcription alongside time-offset metadata.

**Reasoning**:
Import necessary libraries for audio processing and attachment handling, and iterate through the structured output to attempt audio extraction, conversion, transcription, and metadata storage. Given that direct embedded audio extraction from PDFs is complex and not universally supported by common Python libraries, this code will focus on checking for attachments and assuming they might be audio files, then proceed with transcription if found. It will also include placeholder comments for the audio extraction part due to the complexity and lack of standard library support.

In [16]:
# import speech_recognition as sr # No longer needed, using openai-whisper
# from pydub import AudioSegment # Uncomment if pydub is needed for conversion
import os
import PyPDF2
import whisper # Import whisper
from pydub import AudioSegment # Import AudioSegment

# Initialize the speech recognition engine (using a basic recognizer)
# r = sr.Recognizer() # No longer needed

# Load the Whisper model (choose a suitable model size like 'base', 'small', 'medium')
# This might take some time and download the model the first time.
try:
    model = whisper.load_model("base")
    print("Whisper model loaded.")
except Exception as e:
    print(f"Error loading Whisper model: {e}")
    model = None # Set model to None if loading fails


# Directory to save temporary audio files extracted from attachments
temp_audio_dir = "temp_audio"
os.makedirs(temp_audio_dir, exist_ok=True)


for doc_path, doc_data in structured_output.items():
    print(f"Processing audio for document: {doc_path}")

    # Initialize audio list for the document if it doesn't exist
    if 'audio' not in doc_data:
        doc_data['audio'] = []

    # Attempt to extract attachments using PyPDF2
    try:
        with open(doc_path, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            # Correct way to access embedded files in PyPDF2 >= 3.0.0
            if reader.embedded_files:
                print(f"Found embedded files in {doc_path}")
                for f_name, f_data in reader.embedded_files.items():
                    # Check if embedded file is likely an audio file based on filename extension
                    if f_name.lower().endswith(('.mp3', '.wav', '.ogg', '.flac', '.m4a')): # Added .m4a
                        print(f"Potential audio embedded file found: {f_name}")
                        try:
                            audio_data = f_data.get_data()
                            # Save temporary audio file
                            temp_audio_path = os.path.join(temp_audio_dir, f_name)
                            with open(temp_audio_path, 'wb') as af:
                                af.write(audio_data)
                            print(f"Saved potential audio embedded file to {temp_audio_path}")

                            # --- Audio Conversion to WAV 16kHz mono using pydub ---
                            converted_audio_path = None
                            try:
                                audio = AudioSegment.from_file(temp_audio_path)
                                # Convert to 16 kHz, mono
                                audio = audio.set_frame_rate(16000).set_channels(1)
                                converted_audio_path = temp_audio_path + "_converted.wav"
                                audio.export(converted_audio_path, format="wav")
                                print(f"Converted audio to {converted_audio_path}")
                                audio_source_for_transcription = converted_audio_path
                            except Exception as conv_e:
                                print(f"Error converting audio file {f_name}: {conv_e}")
                                # If conversion fails, try transcribing the original file if whisper supports it
                                # Or handle this error appropriately (e.g., skip transcription)
                                audio_source_for_transcription = temp_audio_path
                                converted_audio_path = None # Indicate conversion failed


                            # --- Transcription using Whisper ---
                            transcribed_segments = []
                            if model and os.path.exists(audio_source_for_transcription):
                                try:
                                    print(f"Transcribing audio from {audio_source_for_transcription} using Whisper...")
                                    # Whisper can handle segmentation automatically
                                    result = model.transcribe(audio_source_for_transcription)

                                    # Extract segments from the result
                                    if 'segments' in result:
                                        for segment in result['segments']:
                                            transcribed_segments.append({
                                                'start_time': segment.get('start'),
                                                'end_time': segment.get('end'),
                                                'text': segment.get('text', '').strip(),
                                                'speaker': 'unknown' # Whisper base model doesn't do speaker diarization
                                            })
                                        print("Transcription complete.")
                                    else:
                                         print("Whisper transcription did not return segments.")


                                except Exception as trans_e:
                                    print(f"Error during Whisper transcription of {audio_source_for_transcription}: {trans_e}")
                            elif not model:
                                print("Whisper model not loaded, skipping transcription.")
                            else:
                                print(f"Audio source file not found for transcription: {audio_source_for_transcription}")


                            # Store transcription and metadata in structured_output
                            doc_data['audio'].append({
                                'filename': f_name,
                                'extracted_path': temp_audio_path,
                                'converted_path': converted_audio_path, # Store path to converted file if successful
                                'transcription': transcribed_segments,
                                'metadata': {} # Add any other relevant metadata
                            })

                        except Exception as attach_e:
                            print(f"Error processing embedded file {f_name} in {doc_path}: {attach_e}")

            else:
                print(f"No embedded files found in {doc_path}")

    except PyPDF2.errors.PdfReadError:
         print(f"Could not read {doc_path} with PyPDF2 (possibly encrypted or corrupted). Skipping embedded file extraction.")
    except Exception as pdf_read_e:
         # Catching potential AttributeError for 'embedded_files' if an older PyPDF2 version is present
         if "'PdfReader' object has no attribute 'embedded_files'" in str(pdf_read_e):
             print(f"Error reading PDF {doc_path}: Could not access embedded files. This might be due to the PyPDF2 version or PDF structure. Original error: {pdf_read_e}")
         else:
             print(f"Error reading PDF {doc_path} for embedded files: {pdf_read_e}")

# Note: Cleaning up temporary audio files can be done at the end of the main task.
# import shutil
# shutil.rmtree(temp_audio_dir)

print("\nAudio extraction and transcription subtask completed.")

100%|███████████████████████████████████████| 139M/139M [00:06<00:00, 23.8MiB/s]


Whisper model loaded.
Processing audio for document: /content/T-REC-D.1140-202008-I!!PDF-E.pdf
Error reading PDF /content/T-REC-D.1140-202008-I!!PDF-E.pdf: Could not access embedded files. This might be due to the PyPDF2 version or PDF structure. Original error: 'PdfReader' object has no attribute 'embedded_files'
Processing audio for document: /content/T-REC-E.812-202009-I!Amd1!PDF-E.pdf
Error reading PDF /content/T-REC-E.812-202009-I!Amd1!PDF-E.pdf: Could not access embedded files. This might be due to the PyPDF2 version or PDF structure. Original error: 'PdfReader' object has no attribute 'embedded_files'
Processing audio for document: /content/T-REC-E.812-202005-I!!PDF-E.pdf
Error reading PDF /content/T-REC-E.812-202005-I!!PDF-E.pdf: Could not access embedded files. This might be due to the PyPDF2 version or PDF structure. Original error: 'PdfReader' object has no attribute 'embedded_files'
Processing audio for document: /content/T-REC-D.609-202502-P!!PDF-E.pdf
Error reading PDF /c

## Combine and structure output

### Subtask:
Combine the extracted text, tables, and image metadata for each page or document into a structured format (e.g., JSON or a custom object).

**Reasoning**:
Combine the extracted text blocks, potential headings, tables, and image metadata for each page and document into a structured dictionary format, as required by the subtask.

In [17]:
structured_output = {}

for doc_path, pages_data in page_info.items():
    structured_output[doc_path] = {
        "document_path": doc_path,
        "pages": []
    }
    for page_data in pages_data:
        structured_page_data = {
            "page_number": page_data.get("page_number"),
            "text_blocks": page_data.get("text_blocks", []),
            "potential_headings": page_data.get("potential_headings", []),
            "tables": page_data.get("tables", []),
            "images": page_data.get("images", [])
        }
        structured_output[doc_path]["pages"].append(structured_page_data)

# Print a sample of the structured output
import json

# Select a document to display (e.g., the first one)
if structured_output:
    first_doc_path = list(structured_output.keys())[0]
    print(f"Sample Structured Output for {first_doc_path}:")
    # Print a sample of the first few pages
    sample_output = structured_output[first_doc_path]
    sample_output["pages"] = sample_output["pages"][:2] # Limit to first 2 pages for sample
    print(json.dumps(sample_output, indent=2))
else:
    print("No structured output generated.")

Sample Structured Output for /content/T-REC-D.1140-202008-I!!PDF-E.pdf:
{
  "document_path": "/content/T-REC-D.1140-202008-I!!PDF-E.pdf",
  "pages": [
    {
      "page_number": 1,
      "text_blocks": [
        "I n t e r n a t i o n a l T e l e c o m m u n i c a t i o n U n i o n ITU-T D.1140/X.1261 TELECOMMUNICATION STANDARDIZATION SECTOR OF ITU (08/2020) SERIES D: TARIFF AND ACCOUNTING PRINCIPLES AND INTERNATIONAL TELECOMMUNICATION/ICT ECONOMIC AND POLICY ISSUES Recommendations for international telecommunication/ICT economic and policy issues \u2013 Economic and policy aspects of big data and digital identity in international telecommunications services and networks SERIES X: DATA NETWORKS, OPEN SYSTEM COMMUNICATIONS AND SECURITY Cyberspace security \u2013 Identity management Policy framework including principles for digital identity infrastructure Recommendation ITU-T D.1140/X.1261"
      ],
      "potential_headings": [],
      "tables": [],
      "images": [
        {
         

## Embedded audio extraction & transcription

### Subtask:
Extract embedded audio streams from PDF (if supported) or from attachments. Convert audio to a uniform format (e.g., WAV, 16 kHz mono). Transcribe with a speech-to-text engine (e.g., Whisper). Segment transcription by speaker or time-chunk (e.g., every 15 sec). Store transcription alongside time-offset metadata.

**Reasoning**:
Import necessary libraries for audio processing and attachment handling, and iterate through the structured output to attempt audio extraction, conversion, transcription, and metadata storage. Given that direct embedded audio extraction from PDFs is complex and not universally supported by common Python libraries, this code will focus on checking for attachments and assuming they might be audio files, then proceed with transcription if found. It will also include placeholder comments for the audio extraction part due to the complexity and lack of standard library support.

In [ ]:
import whisper # Import whisper
from pydub import AudioSegment # Import AudioSegment
import PyPDF2
import os
import json

# Load the Whisper model (choose a suitable model size like 'base', 'small', 'medium')
# This might take some time and download the model the first time.
try:
    model = whisper.load_model("base")
    print("Whisper model loaded.")
except Exception as e:
    print(f"Error loading Whisper model: {e}")
    model = None # Set model to None if loading fails


# Directory to save temporary audio files extracted from attachments
temp_audio_dir = "temp_audio"
os.makedirs(temp_audio_dir, exist_ok=True)


for doc_path, doc_data in structured_output.items():
    print(f"Processing audio for document: {doc_path}")

    # Initialize audio list for the document if it doesn't exist
    if 'audio' not in doc_data:
        doc_data['audio'] = []

    # Attempt to extract attachments using PyPDF2
    try:
        with open(doc_path, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            # Correct way to access embedded files in PyPDF2 >= 3.0.0
            if reader.embedded_files:
                print(f"Found embedded files in {doc_path}")
                for f_name, f_data in reader.embedded_files.items():
                    # Check if embedded file is likely an audio file based on filename extension
                    if f_name.lower().endswith(('.mp3', '.wav', '.ogg', '.flac', '.m4a')): # Added .m4a
                        print(f"Potential audio embedded file found: {f_name}")
                        try:
                            audio_data = f_data.get_data()
                            # Save temporary audio file
                            temp_audio_path = os.path.join(temp_audio_dir, f_name)
                            with open(temp_audio_path, 'wb') as af:
                                af.write(audio_data)
                            print(f"Saved potential audio embedded file to {temp_audio_path}")

                            # --- Audio Conversion to WAV 16kHz mono using pydub ---
                            converted_audio_path = None
                            try:
                                audio = AudioSegment.from_file(temp_audio_path)
                                # Convert to 16 kHz, mono
                                audio = audio.set_frame_rate(16000).set_channels(1)
                                converted_audio_path = temp_audio_path + "_converted.wav"
                                audio.export(converted_audio_path, format="wav")
                                print(f"Converted audio to {converted_audio_path}")
                                audio_source_for_transcription = converted_audio_path
                            except Exception as conv_e:
                                print(f"Error converting audio file {f_name}: {conv_e}")
                                # If conversion fails, try transcribing the original file if whisper supports it
                                # Or handle this error appropriately (e.g., skip transcription)
                                audio_source_for_transcription = temp_audio_path
                                converted_audio_path = None # Indicate conversion failed


                            # --- Transcription using Whisper ---
                            transcribed_segments = []
                            if model and os.path.exists(audio_source_for_transcription):
                                try:
                                    print(f"Transcribing audio from {audio_source_for_transcription} using Whisper...")
                                    # Whisper can handle segmentation automatically
                                    result = model.transcribe(audio_source_for_transcription)

                                    # Extract segments from the result
                                    if 'segments' in result:
                                        for segment in result['segments']:
                                            transcribed_segments.append({
                                                'start_time': segment.get('start'),
                                                'end_time': segment.get('end'),
                                                'text': segment.get('text', '').strip(),
                                                'speaker': 'unknown' # Whisper base model doesn't do speaker diarization
                                            })
                                        print("Transcription complete.")
                                    else:
                                         print("Whisper transcription did not return segments.")


                                except Exception as trans_e:
                                    print(f"Error during Whisper transcription of {audio_source_for_transcription}: {trans_e}")
                            elif not model:
                                print("Whisper model not loaded, skipping transcription.")
                            else:
                                print(f"Audio source file not found for transcription: {audio_source_for_transcription}")


                            # Store transcription and metadata in structured_output
                            doc_data['audio'].append({
                                'filename': f_name,
                                'extracted_path': temp_audio_path,
                                'converted_path': converted_audio_path, # Store path to converted file if successful
                                'transcription': transcribed_segments,
                                'metadata': {} # Add any other relevant metadata
                            })

                        except Exception as attach_e:
                            print(f"Error processing embedded file {f_name} in {doc_path}: {attach_e}")

            else:
                print(f"No embedded files found in {doc_path}")

    except PyPDF2.errors.PdfReadError:
         print(f"Could not read {doc_path} with PyPDF2 (possibly encrypted or corrupted). Skipping embedded file extraction.")
    except Exception as pdf_read_e:
         # Catching potential AttributeError for 'embedded_files' if an older PyPDF2 version is present
         if "'PdfReader' object has no attribute 'embedded_files'" in str(pdf_read_e):
             print(f"Error reading PDF {doc_path}: Could not access embedded files. This might be due to the PyPDF2 version or PDF structure. Original error: {pdf_read_e}")
         else:
             print(f"Error reading PDF {doc_path} for embedded files: {pdf_read_e}")

# Note: Cleaning up temporary audio files can be done at the end of the main task.
# import shutil
# shutil.rmtree(temp_audio_dir)

print("\nAudio extraction and transcription subtask completed.")

Whisper model loaded.

Audio extraction and transcription subtask completed.


## Chunking for Embedding

### Subtask:
For each modality (text, table, image, audio), break content into semantically coherent chunks (<1,000 tokens for text; one table per chunk; one image or transcription segment per chunk).

Text chunks: 300–500 words, with 50‑word overlap.

Table chunks: one table = one chunk + its summary.

Image chunks: one chunk per extracted image + its OCRed/text caption.

Audio chunks: one transcription segment = one chunk.

**Reasoning**:
Iterate through the structured output and create chunks for each modality (text, tables, images, audio) based on the specified rules. For text, implement a sliding window approach with overlap. For tables, images, and audio, treat each item as a separate chunk.

In [18]:
# This is a simplified chunking implementation.
# A more sophisticated approach might use tokenizers to control chunk size more precisely
# and handle different languages or text structures.

def chunk_text(text_blocks, chunk_size_words=400, overlap_words=50):
    """Chunks text blocks into overlapping segments of a specified word count."""
    chunks = []
    all_text = " ".join(text_blocks)
    words = all_text.split()
    total_words = len(words)

    if total_words <= chunk_size_words:
        if words:
            chunks.append(" ".join(words))
        return chunks

    step_size = chunk_size_words - overlap_words
    for i in range(0, total_words, step_size):
        chunk = " ".join(words[i:i + chunk_size_words])
        chunks.append(chunk)

    return chunks

chunked_output = {}

for doc_path, doc_data in structured_output.items():
    chunked_output[doc_path] = {
        "document_path": doc_path,
        "chunks": []
    }

    # Process Text Chunks
    all_doc_text_blocks = []
    for page_data in doc_data.get("pages", []):
        all_doc_text_blocks.extend(page_data.get("text_blocks", []))

    text_chunks = chunk_text(all_doc_text_blocks)
    for i, chunk in enumerate(text_chunks):
        # Find the page number(s) associated with this text chunk.
        # This is a simplification; a more accurate approach would track
        # the origin page of each word or sentence.
        # For now, we'll just note that the chunk is from the text modality.
        chunked_output[doc_path]["chunks"].append({
            "content": chunk,
            "modality": "text",
            "source_document": doc_path,
            "chunk_index": i,
            "metadata": {} # Metadata will be added in the next step
        })

    # Process Table Chunks
    for page_data in doc_data.get("pages", []):
        for i, table_info in enumerate(page_data.get("tables", [])):
            # Each table (data + summary) is a chunk
            chunk_content = {
                "table_data": table_info.get("data", []),
                "table_summary": table_info.get("summary", "")
            }
            chunked_output[doc_path]["chunks"].append({
                "content": chunk_content,
                "modality": "table",
                "source_document": doc_path,
                "page_number": page_data.get("page_number"), # Add page number for tables
                "table_index_on_page": i,
                "metadata": {} # Metadata will be added in the next step
            })

    # Process Image Chunks
    for page_data in doc_data.get("pages", []):
        for i, image_info in enumerate(page_data.get("images", [])):
            # Each image is a chunk
            chunked_output[doc_path]["chunks"].append({
                "content": image_info, # Store image metadata as content for now
                "modality": "image",
                "source_document": doc_path,
                "page_number": page_data.get("page_number"), # Add page number for images
                "image_index_on_page": i,
                "metadata": {} # Metadata will be added in the next step
            })

    # Process Audio Chunks (from transcription segments)
    # Assuming audio is at the document level for now based on previous step's structure
    for audio_info in doc_data.get("audio", []):
        for i, segment in enumerate(audio_info.get("transcription", [])):
            # Each transcription segment is a chunk
            chunked_output[doc_path]["chunks"].append({
                "content": segment.get("text", ""), # Store transcription text as content
                "modality": "audio",
                "source_document": doc_path,
                "audio_filename": audio_info.get("filename"),
                "segment_index": i,
                "start_time": segment.get("start_time"), # Include time offsets
                "end_time": segment.get("end_time"),
                "metadata": {} # Metadata will be added in the next step
            })


# Print a sample of the chunked output
print("\nSample Chunked Output:")
if chunked_output:
    first_doc_path = list(chunked_output.keys())[0]
    print(f"Sample Chunked Output for {first_doc_path}:")
    # Print the first few chunks for the first document
    sample_chunks = chunked_output[first_doc_path]["chunks"][:5] # Limit to first 5 chunks
    for i, chunk in enumerate(sample_chunks):
        print(f"  Chunk {i+1} (Modality: {chunk['modality']}):")
        if chunk['modality'] == 'text':
            print(f"    Content (first 200 chars): {chunk['content'][:200]}...")
        elif chunk['modality'] == 'table':
            print(f"    Summary: {chunk['content']['table_summary']}")
            print(f"    Sample Data: {json.dumps(chunk['content']['table_data'][:1], indent=2)}...") # Print first row of data
        elif chunk['modality'] == 'image':
             print(f"    Image File: {chunk['content'].get('file_path')}, Classification: {chunk['content'].get('classification')}")
        elif chunk['modality'] == 'audio':
             print(f"    Transcription (first 200 chars): {chunk['content'][:200]}...")
             print(f"    Time: {chunk.get('start_time')} - {chunk.get('end_time')}")

else:
    print("No chunked output generated.")


Sample Chunked Output:
Sample Chunked Output for /content/T-REC-D.1140-202008-I!!PDF-E.pdf:
  Chunk 1 (Modality: text):
    Content (first 200 chars): I n t e r n a t i o n a l T e l e c o m m u n i c a t i o n U n i o n ITU-T D.1140/X.1261 TELECOMMUNICATION STANDARDIZATION SECTOR OF ITU (08/2020) SERIES D: TARIFF AND ACCOUNTING PRINCIPLES AND INTER...
  Chunk 2 (Modality: text):
    Content (first 200 chars): to the African Region D.600–D.699 Recommendations applicable to the Arab Region D.700–D.799 Recommendations applicable to the Eastern Europe, Central Asia and Transcaucasia Region D.800–D.899 RECOMMEN...
  Chunk 3 (Modality: table):
    Summary: Table 1: A table with 3 rows and 4 columns. Columns include: TERMS AND DEFINITIONS D.0 GENERAL TARIFF PRINCIPLES Private leased telecommunication facilities D.1–D.9 Tariff principles applying to data communication services over dedicated public data networks D.10–D.39 Charging and accounting in the international public telegram service D

## Metadata Enrichment

### Subtask:
For each chunk, attach a metadata object including document ID, page, modality, section heading, chunk ID, timestamp, and additional tags like keywords, document series, and publication date.

**Reasoning**:
Iterate through the chunked output and add a metadata dictionary to each chunk. Populate the metadata with relevant information extracted during the previous steps, including document details, page number (where applicable), modality, and generate a unique chunk ID. Placeholder for keyword extraction and other document-level metadata will be included as these require additional processing or access to document properties not fully covered yet.

In [19]:
from datetime import datetime

# Function to generate a unique chunk ID (example format)
def generate_chunk_id(doc_path, page_number, modality, index_on_page):
    doc_name = os.path.basename(doc_path).replace(".", "_")
    return f"{doc_name}_p{page_number}_{modality}_{index_on_page:04d}"

# Placeholder for keyword extraction (requires additional libraries/models like spaCy or KeyBERT)
def extract_keywords(text):
    # This is a basic placeholder. A real implementation would use NLP tools.
    words = text.split()
    # Return the most frequent words as a simple example of keywords
    word_counts = {}
    for word in words:
        word = word.lower().strip('.,!?"\'()[]{}') # Basic cleaning
        if word:
            word_counts[word] = word_counts.get(word, 0) + 1
    # Sort by frequency and return top N (e.g., top 5)
    sorted_keywords = sorted(word_counts.items(), key=lambda item: item[1], reverse=True)
    return [word for word, count in sorted_keywords[:5]] # Return top 5 words

# Function to extract document-level metadata (placeholders)
def extract_document_metadata(doc_path):
    # In a real application, this would involve parsing document properties
    # or relying on a structured source of document metadata.
    # For this example, we'll use placeholders.
    doc_series = "Unknown Series"
    publication_date = "Unknown Date" # Use a proper date format if available
    document_id = os.path.basename(doc_path) # Using filename as a simple ID

    # Example: Try to parse date from filename if it follows a pattern
    match_date = re.search(r'\d{8}', os.path.basename(doc_path))
    if match_date:
        try:
            publication_date = datetime.strptime(match_date.group(0), '%Y%m%d').isoformat()
        except ValueError:
            pass # Keep "Unknown Date" if parsing fails

    # Example: Try to extract series from filename
    match_series = re.match(r'(T-REC-[A-Z]\.\d+)', os.path.basename(doc_path))
    if match_series:
        doc_series = match_series.group(1)


    return {
        "document_id": document_id,
        "document_series": doc_series,
        "publication_date": publication_date,
        # Add other document-level metadata here
    }


enriched_chunked_output = {}

for doc_path, doc_data in chunked_output.items():
    enriched_chunked_output[doc_path] = {
        "document_path": doc_path,
        "chunks": []
    }
    # Extract document-level metadata once per document
    document_metadata = extract_document_metadata(doc_path)


    for chunk in doc_data["chunks"]:
        metadata = {
            "document_id": document_metadata["document_id"],
            "page": chunk.get("page_number"), # Page number is already in chunk data for some modalities
            "modality": chunk["modality"],
            # "section_heading": "Need to implement section heading mapping", # Requires mapping chunks to sections
            "chunk_id": "", # Will be generated below
            "timestamp": datetime.now().isoformat(), # Example timestamp
            "document_series": document_metadata["document_series"],
            "publication_date": document_metadata["publication_date"],
            "keywords": [], # Placeholder for extracted keywords
             "source_document": chunk.get("source_document") # Add source document path
        }

        # Generate chunk_id based on modality and index within that modality type for the document
        # This requires re-indexing chunks by modality within each document
        # A simpler approach for now: use a global index across all chunks for a document
        # Or re-process the original structured_output to get page and index information more reliably
        # Let's use a simple sequential index for now and refine if needed.
        # The previous chunking code already added index_on_page for tables and images.
        # Need to handle text and audio indexing.

        chunk_index_in_modality = -1 # Initialize index

        if chunk["modality"] == "text":
            # The chunk_index from chunk_text is sequential for text chunks
            chunk_index_in_modality = chunk.get("chunk_index", 0)
             # For text, page number is more complex to determine accurately per chunk
            metadata["page"] = "See text block sources" # Indicate page info is aggregated


        elif chunk["modality"] == "table":
             chunk_index_in_modality = chunk.get("table_index_on_page", 0)


        elif chunk["modality"] == "image":
             chunk_index_in_modality = chunk.get("image_index_on_page", 0)


        elif chunk["modality"] == "audio":
            chunk_index_in_modality = chunk.get("segment_index", 0)
             # Audio is document-level for now, no specific page number
            metadata["page"] = "Document Level"


        # Generate the chunk ID
        if metadata["page"] is not None: # Only generate if page info is somewhat available
             # Use a placeholder page number like 0 if it's document level for ID generation
             page_for_id = metadata["page"] if isinstance(metadata["page"], int) else 0
             metadata["chunk_id"] = generate_chunk_id(doc_path, page_for_id, chunk["modality"], chunk_index_in_modality)
        else:
             # Generate a simpler ID if page info is missing
             metadata["chunk_id"] = f"{document_metadata['document_id']}_{chunk['modality']}_{chunk_index_in_modality:04d}"


        # Add extracted keywords for text and audio content
        if chunk["modality"] in ["text", "audio"] and isinstance(chunk["content"], str):
             metadata["keywords"] = extract_keywords(chunk["content"])
        elif chunk["modality"] == "table" and isinstance(chunk["content"], dict) and "table_summary" in chunk["content"]:
             # Extract keywords from table summary
             metadata["keywords"] = extract_keywords(chunk["content"]["table_summary"])
        # Keywords for images might come from captions if available, or image classification


        # Attach the metadata to the chunk
        chunk["metadata"] = metadata
        enriched_chunked_output[doc_path]["chunks"].append(chunk)


# Print a sample of the enriched chunked output
print("\nSample Enriched Chunked Output:")
if enriched_chunked_output:
    first_doc_path = list(enriched_chunked_output.keys())[0]
    print(f"Sample Enriched Chunked Output for {first_doc_path}:")
    # Print the first few enriched chunks for the first document
    sample_chunks = enriched_chunked_output[first_doc_path]["chunks"][:5] # Limit to first 5 chunks
    for i, chunk in enumerate(sample_chunks):
        print(f"  Chunk {i+1} (Modality: {chunk['modality']}):")
        print(f"    Chunk ID: {chunk['metadata'].get('chunk_id')}")
        print(f"    Page: {chunk['metadata'].get('page')}")
        print(f"    Keywords: {chunk['metadata'].get('keywords')}")
        print(f"    Source Document: {chunk['metadata'].get('source_document')}")
        # print(f"    Metadata: {json.dumps(chunk['metadata'], indent=2)}") # Uncomment to see full metadata
        if chunk['modality'] == 'text':
            print(f"    Content (first 100 chars): {chunk['content'][:100]}...")
        elif chunk['modality'] == 'table':
            print(f"    Summary: {chunk['content']['table_summary']}")
        elif chunk['modality'] == 'image':
             print(f"    Image File: {chunk['content'].get('file_path')}")
        elif chunk['modality'] == 'audio':
             print(f"    Transcription (first 100 chars): {chunk['content'][:100]}...")


Sample Enriched Chunked Output:
Sample Enriched Chunked Output for /content/T-REC-D.1140-202008-I!!PDF-E.pdf:
  Chunk 1 (Modality: text):
    Chunk ID: T-REC-D_1140-202008-I!!PDF-E_pdf_p0_text_0000
    Page: See text block sources
    Keywords: ['and', 'international', 'accounting', 'the', 'charging']
    Source Document: /content/T-REC-D.1140-202008-I!!PDF-E.pdf
    Content (first 100 chars): I n t e r n a t i o n a l T e l e c o m m u n i c a t i o n U n i o n ITU-T D.1140/X.1261 TELECOMMUN...
  Chunk 2 (Modality: text):
    Chunk ID: T-REC-D_1140-202008-I!!PDF-E_pdf_p0_text_0001
    Page: See text block sources
    Keywords: ['and', 'of', 'services', 'to', 'the']
    Source Document: /content/T-REC-D.1140-202008-I!!PDF-E.pdf
    Content (first 100 chars): to the African Region D.600–D.699 Recommendations applicable to the Arab Region D.700–D.799 Recommen...
  Chunk 3 (Modality: table):
    Chunk ID: T-REC-D_1140-202008-I!!PDF-E_pdf_p2_table_0000
    Page: 2
    Keywords: ['and', 'i